In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Selecting 6 lables randomly form 20 newsgroups dataset
lab = np.random.random_integers(0, 19, 6).tolist()
categories = ['comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',              'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball',              'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space',              'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc',              'talk.religion.misc', 'alt.atheism']


In [ ]:
lab.sort()
cats = [categories[i] for i in lab]
cats

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
newsgroups = fetch_20newsgroups(categories=cats, remove=('headers', 'footers', 'quotes'))

In [ ]:
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

docs = [preprocess(text) for text in newsgroups.data]


In [ ]:
tagged_data = [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(docs)]
model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=1, workers=4, epochs=20)
document_vectors = [model.infer_vector(text.split()) for text in docs]

In [ ]:
import pandas as pd
doc_names = ['Doc {}'.format(i) for i in range(len(docs))]
lables = [cats[lab] for lab in list(newsgroups.target)]
df = pd.DataFrame()
df['Docs'] = doc_names
df['vec'] = document_vectors
df['lables'] = lables
df['numLables'] = newsgroups.target

In [ ]:
df.head()

## Computing The Dissimilarity

In [ ]:
import sklearn
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def getCosSim(vec1, vec2) : 
  return cosine_similarity(vec1.reshape(1, -1), vec2.reshape(1, -1))[0][0]

In [ ]:
def getDissimMatrix(df) : 
  n = df.shape[0]
  res = np.zeros((n, n))
  for i in range(n) :
    for j in range(i + 1, n) : 
      res[i, j] = res[j, i] = 1 - getCosSim(df.iloc[i]['vec'], df.iloc[j]['vec'])
    print('Node', i, 'completed')
  return res

In [ ]:
disMat = getDissimMatrix(df)

In [ ]:
disMat_df = pd.DataFrame(disMat, columns = df.index, index = df.index)
disMat_df

## Clustering

In [ ]:
!pip install cvxpy
import numpy as np
import networkx as nx
from scipy.sparse import csr_matrix
from scipy.linalg import sqrtm
import cvxpy as cp
from sklearn import preprocessing as pp
import time


In [ ]:
class BinaryQP(object) : 
    def __init__(self, adjacency) : 
        
        self.adj = adjacency
        self.n = len(self.adj)
        
        self.pot = np.array([0 for i in range(self.n)])
        self.constant = 0
        n = self.n
        
        cost_matrix = np.zeros((n+1, n+1))
        cost_matrix[np.ix_(list(range(n)),list(range(n)))] = self.adj
        cost_matrix[np.ix_([n],list(range(n)))] = [i/2 for i in self.pot]
        cost_matrix[np.ix_(list(range(n))),[n]] = [i/2 for i in self.pot]
        cost_matrix[n][n] = self.constant
        
        self.cost_matrix = cost_matrix
        
    def __str__(self) : 
        print("The Cost Matrix (S) is :")
        return str(self.cost_matrix)

In [ ]:
def solveUsingEPSDP(problem, rank,la, ga, steps):
    C = problem.cost_matrix
    D = np.diag(np.sum(C, axis=0))
    L = D - C
    C = L
    V = np.random.normal(0, 1, (rank, len(C)))
    V = np.transpose(pp.normalize(np.transpose(V), norm='l2'))
    step = 1/np.linalg.norm(C) 
    
    for outer_steps in range(10):
    # while np.trace(np.dot(V.T, np.dot(V, C))) - la*np.max(np.abs(V)) > 1e-6:
        # if outerSteps > 100 : break
        # print(outer_steps, 'iteration')
        gradient = 2*np.matmul(V, C) - la*TsallisGrad(V)
        innerSteps = 0 
        # while np.trace(gradient) > 1e-6 :
        #     if innerSteps > 1000 : break
        for s in range(steps):
            gradient = 2*np.matmul(V, C) - la*TsallisGrad(V)
            V = V + step*(gradient)
            V = np.transpose(pp.normalize(np.transpose(V), norm='l2'))
            innerSteps += 1
        
        la = la*ga
        # outerSteps += 1
    U, s, Vt = np.linalg.svd(V)
    v1 = Vt.T[:, 0]
    return v1



def TsallisGrad(V):
    alpha = 3
    U1, d, U2 = np.linalg.svd(V, full_matrices=False)
    D = np.diag(d)    
    return (alpha/(1-alpha))*(np.dot(U1, np.dot(np.linalg.matrix_power(D, 2*alpha-1),U2))/np.power(np.trace(np.dot(D, D)), alpha) -  np.trace(np.linalg.matrix_power(D, 2*alpha))*V/np.power(np.trace(np.dot(D, D)), alpha+1) )

In [ ]:
n = disMat.shape[0]
def getAdj(nodes, disMat_df) : 
  n = len(nodes)
  return disMat_df.iloc[nodes, nodes].to_numpy()
  

  # adj = np.zeros((n, n))
  # # print(adj.shape)
  # for ind1, node1 in enumerate(nodes) : 
  #   for ind2, node2 in enumerate(nodes) : 
  #     adj[ind1, ind2] = disMat_df.iloc[node1, node2]
  #   print('node ', ind1)
  # return adj

In [ ]:
def getLables(problem : BinaryQP) : 
  res = solveUsingEPSDP(problem, rank = 20, la = 10, ga = 5, steps = 100)
  n = problem.n
  r = np.random.randn(n + 1)
  lables = list(np.sign(res))
  return lables

In [ ]:
class ClusterNode(object) :
  def __init__(self, items : list, level : int, config : list) : 
    self.items = items
    self.level = level
    self.config = config
    self.left = None
    self.right = None
  
  def __str__(self) : 
    res = 'items : ' + str(self.items) + '\n'
    res += 'level : ' + str(self.level) + '\n'
    res += 'config : ' + str(self.config) + '\n'
    return res


In [ ]:
max_clusters = 10

In [ ]:
def getWCSS(node : ClusterNode) : 
  items = node.items
  centroid = np.zeros(df.loc[0, 'vec'].shape[0])
  for item in items : 
    centroid += df.loc[item, 'vec']
  centroid = centroid / len(items)
  res = 0 
  for item in items : 
    res += np.linalg.norm(centroid - df.loc[item, 'vec'])
  return res 


In [ ]:
from heapq import heappush, heappop
treeMemo = dict()
wcssMemo = dict()



def divisiveCluster(disMat_df, leafLength) -> ClusterNode : 
  n = disMat_df.shape[0]
  root = ClusterNode (
      items = list(range(n)),
      level = 0,
      config = []
  )

  q = [(0, root)]

  while len(q) < max_clusters : 
    treeMemo[len(q)] = q.copy()
    # total_WCSS = sum([-(i* len(j.items)) for i, j in q])
    total_WCSS = sum([-i for i, j in q])
    wcssMemo[len(q)] = total_WCSS / len(q)
    _, currNode = q.pop(0)
    if len(currNode.items) < leafLength : continue
    # print('Cluster at level : ', currNode.level, 'is divided into ', end = " ")
    adj = getAdj(currNode.items, disMat_df)
    problem = BinaryQP(adj)
    lables = getLables(problem)
    pos =  []
    neg = []
    for item, lable in zip(currNode.items, lables) : 
      if lable > 0 : pos.append(item)
      else : neg.append(item)

    leftCluster = ClusterNode(items = neg, level = currNode.level + 1, config = currNode.config + [-1])
    rightCluster = ClusterNode(items = pos, level = currNode.level+ 1, config = currNode.config + [1] )
    currNode.left = leftCluster
    currNode.right = rightCluster 
    heappush(q, (-getWCSS(leftCluster), leftCluster))
    heappush(q, (-getWCSS(rightCluster), rightCluster))

    print('Cluster at level '+ str(currNode.level) +  ' with ' +  str(len(currNode.items)) + ' divided into ' + str( len(currNode.left.items))  +' and ' + str(len(currNode.right.items)) + ' nodes.'   )
    print(currNode)



  return root

In [ ]:
rootCluster = divisiveCluster(disMat_df,20)

In [ ]:
def printTree(root) : 
  if not root : return
  q = [root]
  while q : 
    node = q.pop(0)
    if node == None : print(' N ', end = ' ')
    else : 
      print(' NODE ', end = ' ')
      q.append(node.left)
      q.append(node.right)

In [ ]:
printTree(rootCluster)

In [ ]:
import matplotlib.pyplot as plt
del wcssMemo[1]
plt.figure(figsize=(10, 10))
plt.plot(wcssMemo.keys(), wcssMemo.values())
plt.show()

In [ ]:
import numpy as np
from scipy.signal import savgol_filter

def find_optimal_num_clusters(wcss, num_clusters):
    diff1 = np.diff(wcss)
    diff1_smooth = savgol_filter(diff1, window_length=7, polyorder=2, mode='nearest')
    diff2 = np.diff(diff1_smooth)
    optimal_num_clusters = num_clusters[np.argmax(diff2) + 1]
    return optimal_num_clusters



wcss = np.array(list(wcssMemo.values()))
num_clusters = np.array(list(wcssMemo.keys()))
optimal_num_clusters = find_optimal_num_clusters(wcss, num_clusters)
print("Optimal number of clusters:", optimal_num_clusters)


In [ ]:
clusters = treeMemo[optimal_num_clusters]

In [ ]:
memo = dict()
for id, _ in enumerate(clusters) : 
  _, cluster = _
  for item in cluster.items : 
    memo[item] = id

In [ ]:
vals = sorted(memo.items())
vals = [i for j, i in vals]

In [ ]:
df['pred'] = vals
df

In [ ]:
true_labels = df['numLables']
predicted_labels = df['pred']

In [ ]:
import numpy as np
from sklearn.metrics import jaccard_score

n_instances = true_labels.shape[0]
n_clusters = len(np.unique(predicted_labels.values))

# Create contingency table
contingency_table = np.zeros((n_clusters, n_instances))
for i in range(n_instances):
    contingency_table[predicted_labels[i], true_labels[i]] += 1

# Calculate Jaccard coefficients
jaccard_coefficients = []
for i in range(n_clusters):
    jaccard_coefficients.append(jaccard_score(contingency_table[i], contingency_table.max(axis=0), average = 'macro'))

# Map clusters to ground truth labels
cluster_to_ground_truth = np.argmax(contingency_table, axis=1)
cluster_to_ground_truth = np.array([cluster_to_ground_truth[i] for i in range(n_clusters)])


In [ ]:
mappedLables = cluster_to_ground_truth.tolist() 
mappedLables

In [ ]:
df['mapped'] = df['pred'].map(lambda x : mappedLables[x])
df.head()

In [ ]:
from sklearn.metrics.cluster import rand_score
rand_score(df['numLables'].values,df['mapped'].values)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df['numLables'].values, df['mapped'].values))
cp = classification_report(df['numLables'].values, df['mapped'].values)

In [ ]:
text_file = open(' '.join(cats) + ' 1.txt', "w")
n = text_file.write(' '.join(cats) + '\n')
n = text_file.write(cp)
text_file.close()

# Hard Code Number of clusters

In [ ]:
clusters = treeMemo[6]

In [ ]:
memo = dict()
for id, _ in enumerate(clusters) : 
  _, cluster = _
  for item in cluster.items : 
    memo[item] = id

In [ ]:
vals = sorted(memo.items())
vals = [i for j, i in vals]

In [ ]:
df['pred'] = vals
df

In [ ]:
true_labels = df['numLables']
predicted_labels = df['pred']

In [ ]:
import numpy as np
from sklearn.metrics import jaccard_score

n_instances = true_labels.shape[0]
n_clusters = len(np.unique(predicted_labels.values))

# Create contingency table
contingency_table = np.zeros((n_clusters, n_instances))
for i in range(n_instances):
    contingency_table[predicted_labels[i], true_labels[i]] += 1

# Calculate Jaccard coefficients
jaccard_coefficients = []
for i in range(n_clusters):
    jaccard_coefficients.append(jaccard_score(contingency_table[i], contingency_table.max(axis=0), average = 'macro'))

# Map clusters to ground truth labels
cluster_to_ground_truth = np.argmax(contingency_table, axis=1)
cluster_to_ground_truth = np.array([cluster_to_ground_truth[i] for i in range(n_clusters)])


In [ ]:
mappedLables = cluster_to_ground_truth.tolist() 
mappedLables

In [ ]:
df['mapped'] = df['pred'].map(lambda x : mappedLables[x])
df.head()

In [ ]:
from sklearn.metrics.cluster import rand_score
rand_score(df['numLables'].values,df['mapped'].values)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df['numLables'].values, df['mapped'].values))
cp = classification_report(df['numLables'].values, df['mapped'].values)

In [ ]:
text_file = open(' '.join(cats) + ' 2.txt', "w")
n = text_file.write(' '.join(cats) + '\n')
n = text_file.write(cp)
text_file.close()

In [ ]:
# Mapping with highest number of nodes with clusters numbers


In [ ]:
memo = dict()
for label in range(20) : 
  memo[label] = df[df['numLables'] == label].index.tolist()

In [ ]:
jacMemo = dict() # label, clusterInd -> sim
for clusterInd, c in enumerate(clusters) :
  cluster = c[1]
  for label in range(20) :
    s1 = set(memo[label]) 
    s2 = set(cluster.items)
    nume = len(s1.intersection(s2))
    deno = len(s2.union(s1))
    jacMemo[(label, clusterInd)] = nume/ deno
    
# jacMemo

In [ ]:
mapping = dict()  # clusterid -> label
clusterSeen = set()
labelSeen = set()
from heapq import heappush, heappop, heapify
# heap = [(len(_[1].item), -sim, _[0], _[1]) for sim, _ in jacMemo.items()]

heap = []
for _, sim in jacMemo.items() : 
  label, clusterId = _
  heap.append((-len(clusters[clusterId][1].items), -sim, label, clusterId))

heapify(heap)
while len(clusterSeen) < 20 and heap : 
  j, sim , label, clusterId = heappop(heap)
  if label in labelSeen or clusterId in clusterSeen : continue
  mapping[clusterId] = label
  labelSeen.add(label)
  clusterSeen.add(clusterId)
 

In [ ]:
mapping

In [ ]:
df['mapped'] = df['pred'].apply(lambda x : mapping[x])
df

In [ ]:
df['mapped'].value_counts()

In [ ]:
from sklearn.metrics.cluster import rand_score
rand_score(df['numLables'].values,df['mapped'].values)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df['numLables'].values, df['mapped'].values))